# Notebook de mejora

Este notebook es un ejemplo cómo se puede mejorar los hiperparámetros de un algoritmo, y medir la importancia de atributos https://www.kaggle.com/c/house-prices-advanced-regression-techniques/ de la práctica 3 de Inteligencia de Negocio. 

Incluyo el código del notebook anterior, no se explicarán.

Es un ejemplo que es claramente mejorable, se deja a el/la estudiante el mejorarlo para obtener mejores resultados.

In [69]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.impute import KNNImputer


In [70]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv", na_values="NaN") # Definimos na_values para identificar bien los valores perdidos


if 'Id' in train:
    train.drop('Id', axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

test = pd.read_csv("test.csv", na_values="NaN")
test_ids = test.Id
test = test.drop('Id', axis=1)

input_all = pd.concat([train.drop('SalePrice', axis=1), test])

col_cat = list(input_all.select_dtypes(exclude=np.number).columns)

from sklearn.impute import SimpleImputer

# Valores categóricos por el más frecuente
imputer_cat = SimpleImputer(strategy="most_frequent")
imputer_cat.fit(input_all[col_cat])
train[col_cat] = imputer_cat.transform(train[col_cat])
test[col_cat] = imputer_cat.transform(test[col_cat])

# Valores numéricos por KNN
col_num = list(train.select_dtypes(include=np.number).columns)
col_num.remove('SalePrice')

# Configuración del imputador KNN
knn_imputer = KNNImputer(n_neighbors=5)  # Puedes ajustar el número de vecinos según tus necesidades

# Imputación para el conjunto de entrenamiento
train[col_num] = knn_imputer.fit_transform(train[col_num])

# Imputación para el conjunto de prueba
test[col_num] = knn_imputer.transform(test[col_num])

from sklearn.preprocessing import LabelEncoder
labelers = {}
test_l = test.copy()
train_l = train.copy()


y_train = train_l.SalePrice
X_train = train_l.drop('SalePrice', axis=1)

if 'Id' in test_l:
    test_l.drop('Id', axis=1, inplace=True)

X_test = test_l

In [71]:
X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60.0,RL,65.0,8450.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,2.0,2008.0,WD,Normal
1,20.0,RL,80.0,9600.0,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,0.0,0.0,Ex,MnPrv,Shed,0.0,5.0,2007.0,WD,Normal
2,60.0,RL,68.0,11250.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,9.0,2008.0,WD,Normal
3,70.0,RL,60.0,9550.0,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,0.0,0.0,Ex,MnPrv,Shed,0.0,2.0,2006.0,WD,Abnorml
4,60.0,RL,84.0,14260.0,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,0.0,0.0,Ex,MnPrv,Shed,0.0,12.0,2008.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,8.0,2007.0,WD,Normal
1456,20.0,RL,85.0,13175.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,2.0,2010.0,WD,Normal
1457,70.0,RL,66.0,9042.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,GdPrv,Shed,2500.0,5.0,2010.0,WD,Normal
1458,20.0,RL,68.0,9717.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,4.0,2010.0,WD,Normal


In [72]:
y_train

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [73]:
# Define the cross-validation strategy
cv = KFold(n_splits=10, shuffle=True, random_state=124345)

In [74]:
from sklearn.compose import make_column_selector

cat_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_include=np.number)

In [75]:
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler

cat_linear_processor = OneHotEncoder(handle_unknown="ignore")
num_linear_processor = StandardScaler()

linear_preprocessor = make_column_transformer(
    (num_linear_processor, num_selector), (cat_linear_processor, cat_selector)
)
linear_preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000271569BF6A0>)])

In [76]:
cat_tree_processor = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=-1,
    encoded_missing_value=-2,
)

num_tree_processor = SimpleImputer(strategy="mean", add_indicator=True)

tree_preprocessor = make_column_transformer(
    (num_tree_processor, num_selector), (cat_tree_processor, cat_selector)
)
tree_preprocessor

ColumnTransformer(transformers=[('simpleimputer',
                                 SimpleImputer(add_indicator=True),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                ('ordinalencoder',
                                 OrdinalEncoder(encoded_missing_value=-2,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000271569BF6A0>)])

In [77]:
from sklearn.ensemble import RandomForestRegressor

rf_pipeline = make_pipeline(tree_preprocessor, RandomForestRegressor(criterion='squared_error', max_depth=20, random_state=42))
rf_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-2,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000271569BF6A0>)])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, random_state=42))])

In [78]:
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# KNN pipeline
knn_pipeline = make_pipeline(tree_preprocessor, KNeighborsRegressor())

knn_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-2,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000271569BF6A0>)])),
                ('kneighborsregressor', KNeighborsRegressor())])

In [64]:
from sklearn.model_selection import GridSearchCV

gradient_boosting = GradientBoostingRegressor(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth': [3, 4, 5]
       
}

# Realizar la búsqueda en la cuadrícula
tuned_gradient_boosting = GridSearchCV(
    gradient_boosting,
    param_grid=param_grid,
    scoring='neg_mean_squared_log_error',
    cv=cv,
    n_jobs=4  # Utiliza múltiples núcleos para acelerar el proceso si es posible
)
# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a column transformer to handle categorical encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Transform the training data
X_train_transformed = preprocessor.fit_transform(X_train)

# Ajustar el modelo a los datos de entrenamiento
tuned_gradient_boosting.fit(X_train_transformed, y_train)

# Obtener los mejores parámetros
best_params = tuned_gradient_boosting.best_params_
print("Mejores parámetros:", best_params)

Mejores parámetros: {'learning_rate': 0.15, 'max_depth': 4, 'n_estimators': 300}


ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\germa\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\germa\anaconda3\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\germa\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 416, in fit
    X, y = self._validate_data(
  File "C:\Users\germa\anaconda3\lib\site-packages\sklearn\base.py", line 621, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\germa\anaconda3\lib\site-packages\sklearn\utils\valid

In [65]:
gradient_boosting = GradientBoostingRegressor(random_state=42, learning_rate=0.15, max_depth=4, n_estimators=333)


# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(gradient_boosting, X_train_transformed, y_train, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores))
print("Std CV Score: ", np.std(cv_scores))

Cross-validation scores:  [0.11023976 0.15763066 0.10692587 0.12343352 0.15539418 0.14343973
 0.15068717 0.11455281 0.12111826 0.09088976]
Mean CV Score:  0.12743117054040926
Std CV Score:  0.021826651443118118


In [79]:
# GradientBoostingRegressor pipeline
gradient_boosting_pipeline = make_pipeline(
    tree_preprocessor, GradientBoostingRegressor(random_state=42, learning_rate=0.15, max_depth=4, n_estimators=333)
)

gradient_boosting_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-2,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000271569BF6A0>)])),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(learning_rate=0.15, max_depth=4,
                                           n_estimators=333,
                                           random_state=42))])

In [80]:
# XGBoost pipeline
xgboost_pipeline = make_pipeline(
    tree_preprocessor, XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, gamma=0.05, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.5, n_estimators=3000,
                             reg_alpha=0.5, reg_lambda=0.8,
                             subsample=0.5,
                             random_state =42, seed=42)
)

xgboost_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-2,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transfor...
                              feature_types=None, gamma=0.05, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=1.5, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=3000, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [81]:
from sklearn.linear_model import LassoCV

lasso_pipeline = make_pipeline(linear_preprocessor, LassoCV(max_iter=3000))
lasso_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000271569BF6A0>)])),
                ('lassocv', LassoCV(max_iter=3000))])

In [86]:
import lightgbm as lgb

model_lgb = lgb.LGBMRegressor(bagging_fraction = 0.8, bagging_seed=9, seed=42)

param_grid = {
    'n_estimators': [500, 1000, 2000],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [2, 5, 10], 
    'bagging_freq' : [2, 5, 10],
    'feature_fraction' : [0.25, 0.5]
       
}

# Realizar la búsqueda en la cuadrícula
tuned_lgbm = GridSearchCV(
    model_lgb,
    param_grid=param_grid,
    scoring='neg_mean_squared_log_error',
    cv=cv,
    n_jobs=4  # Utiliza múltiples núcleos para acelerar el proceso si es posible
)
# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a column transformer to handle categorical encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Transform the training data
X_train_transformed = preprocessor.fit_transform(X_train)

# Ajustar el modelo a los datos de entrenamiento
tuned_lgbm.fit(X_train_transformed, y_train)

# Obtener los mejores parámetros
best_params = tuned_gradient_boosting.best_params_
print("Mejores parámetros:", best_params)

KeyboardInterrupt: 

In [90]:
lgbm = lgb.LGBMRegressor(bagging_fraction = 0.8, bagging_seed=9, seed=42, objective='regression', n_estimators=300, max_depth=4, learning_rate=0.15)


# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(lgbm, X_train_transformed, y_train, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores))
print("Std CV Score: ", np.std(cv_scores))

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002255 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3419
[LightGBM] [Info] Number of data points in the train set: 1314, number of used features: 184
[LightGBM] [Info] Start training from score 180704.734399
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3399
[LightGBM] [Info] Number of data points in the train set: 1314, number of used features: 186
[LightGBM] [Info] Start training from score 181600.187215
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Cross-validation scores:  [0.12702823 0.16028133 0.12266963 0.13362579 0.14948418 0.1495791
 0.15347637 0.12583934 0.12642112 0.08513014]
Mean CV Score:  0.13335352435796732
Std CV Score:  0.02058579120553745


In [92]:
# LightGBM pipeline
lightgbm_pipeline = make_pipeline(
    tree_preprocessor, lgb.LGBMRegressor(objective='regression', bagging_fraction = 0.8, bagging_seed=9, seed=42, n_estimators=300, max_depth=4, learning_rate=0.15)
)

lightgbm_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-2,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000271569BF6A0>)])),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.8, bagging_seed=9,
                               learning_rate=0.15, max_depth=4,
                               n_estimators=300, objective='regression',
                               seed=42))])

In [93]:
# Collect predictions and errors from each base estimator
def collect_predictions_errors(estimator, X, y):
    predictions = estimator.predict(X)
    errors = y - predictions
    return predictions.reshape(-1, 1), errors.reshape(-1, 1)


In [96]:
# Define the base models
base_models = [
    ('random_forest', rf_pipeline, collect_predictions_errors),
    ('knn', knn_pipeline, collect_predictions_errors),
    ('gradient_boosting', gradient_boosting_pipeline, collect_predictions_errors),
    ('xgboost', xgboost_pipeline, collect_predictions_errors),
    ('lasso', lasso_pipeline),
    ('lightgbm', lightgbm_pipeline, collect_predictions_errors)
]

stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=Ridge())
stacking_regressor

StackingRegressor(estimators=[('random_forest',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('simpleimputer',
                                                                                 SimpleImputer(add_indicator=True),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                                                                ('ordinalencoder',
                                                                                 OrdinalEncoder(encoded_missing_value=-2,
                                                                                                handle_unknown='use_encoded_value',
                                                                                                unkn...
                                                                                                handle_unknown='use_encoded_value',
                                                                                                unknown_value=-1),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000271569BF6A0>)])),
                                               ('lgbmregressor',
                                                LGBMRegressor(bagging_fraction=0.8,
                                                              bagging_seed=9,
                                                              learning_rate=0.15,
                                                              max_depth=4,
                                                              n_estimators=300,
                                                              objective='regression',
                                                              seed=42))]),
                               <function collect_predictions_errors at 0x0000027159FAE4C0>)],
                  final_estimator=Ridge())

In [97]:


# Calculate cross-validated negative mean squared log error
values = cross_val_score(stacking_regressor, X_train, y_train, scoring='neg_mean_squared_log_error', cv=cv, error_score='raise')

# Print the results
print(values)
print(values.mean())

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000950 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3317
[LightGBM] [Info] Number of data points in the train set: 1314, number of used features: 73
[LightGBM] [Info] Start training from score 180903.573059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000350 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3052
[LightGBM] [Info] Number of data points in the train set: 1051, number of used features: 70
[LightGBM] [Info] Start training from score 181097.168411
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001688 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3057
[LightGBM] [Info] Number of data points in the train set: 1051, number of used features: 72
[LightGBM] [Info] Start training from score 180849.429115
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000518 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3043
[LightGBM] [Info] Number of data points in the train set: 1051, number of used features: 71
[LightGBM] [Info] Start training from score 179818.201713
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000943 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3059
[LightGBM] [Info] Number of data points in the train set: 1051, number of used features: 71
[LightGBM] [Info] Start training from score 182608.897241
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[-0.008052   -0.01792829 -0.01276611 -0.01329305 -0.01200936 -0.02396269
 -0.02169182 -0.01293589 -0.01837149 -0.01140819]
-0.015241889604194706


In [98]:
stacking_regressor.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000386 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3425
[LightGBM] [Info] Number of data points in the train set: 1460, number of used features: 73
[LightGBM] [Info] Start training from score 180921.195890
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000652 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3135
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 71
[LightGBM] [Info] Start training from score 180717.091610
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000881 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3148
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 72
[LightGBM] [Info] Start training from score 180407.575342
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000684 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3139
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 72
[LightGBM] [Info] Start training from score 180007.375000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001816 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3156
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 71
[LightGBM] [Info] Start training from score 182883.660103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001551 seconds.
You can set `force_c

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

StackingRegressor(estimators=[('random_forest',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('simpleimputer',
                                                                                 SimpleImputer(add_indicator=True),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000002715A0C07F0>),
                                                                                ('ordinalencoder',
                                                                                 OrdinalEncoder(encoded_missing_value=-2,
                                                                                                handle_unknown='use_encoded_value',
                                                                                                unkn...
                                                                                                handle_unknown='use_encoded_value',
                                                                                                unknown_value=-1),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000271569BF6A0>)])),
                                               ('lgbmregressor',
                                                LGBMRegressor(bagging_fraction=0.8,
                                                              bagging_seed=9,
                                                              learning_rate=0.15,
                                                              max_depth=4,
                                                              n_estimators=300,
                                                              objective='regression',
                                                              seed=42))]),
                               <function collect_predictions_errors at 0x0000027159FAE4C0>)],
                  final_estimator=Ridge())

In [99]:
pred = stacking_regressor.predict(X_test)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


In [100]:
salida = pd.DataFrame({'Id': test_ids, 'SalePrice': pred})
salida.to_csv("Prueba9.csv", index=False)

In [11]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_log_error

# Split the data into training and validation sets
X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a column transformer to handle categorical encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Transform the training data
X_train2_transformed = preprocessor.fit_transform(X_train2)
X_train_transformed = preprocessor.fit_transform(X_train)

# Create an XGBoost regressor
xg_reg = xgb.XGBRegressor(objective='reg:squaredlogerror', seed=42)

# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(xg_reg, X_train2_transformed, y_train2, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores))
print("Std CV Score: ", np.std(cv_scores))

# Fit the model on the entire training set
xg_reg.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = xg_reg.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score)


Cross-validation scores:  [7.46938917 7.44492932 7.41372314 7.46061306 7.47845455 7.46462243
 7.46449019 7.42516991 7.4336704  7.48145291]
Mean CV Score:  7.4536515092289
Std CV Score:  0.02191547665096053
Validation Score (neg_mean_squared_log_error):  52.2259646078938


In [14]:
# Transform the training data
X_train_transformed = preprocessor.fit_transform(X_train)

# Create an XGBoost regressor
xg_reg = xgb.XGBRegressor(colsample_bytree=0.5, gamma=0.05, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.5, n_estimators=3000,
                             reg_alpha=0.5, reg_lambda=0.8,
                             subsample=0.5,
                             random_state =42, seed=42)

# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores_xg = cross_val_score(xg_reg, X_train_transformed, y_train, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores_xg)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores_xg))
print("Std CV Score: ", np.std(cv_scores_xg))

# Fit the model on the entire training set
xg_reg.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = xg_reg.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score)

Cross-validation scores:  [0.11607035 0.1617085  0.10901048 0.12382963 0.16112732 0.15060409
 0.1340738  0.10593501 0.12369194 0.08150088]
Mean CV Score:  0.12675519924564455
Std CV Score:  0.024400665776440306
Validation Score (neg_mean_squared_log_error):  0.00034680805367093693


In [24]:
X_train_final = X_train.copy()

# Add a new column to the training set with predicted values
X_train_final["PredictedValuesXGBoost"] = xg_reg.predict(X_train_transformed)
X_train_final["MeanCVXGBoost"] = np.mean(cv_scores_xg)
X_train_final["StdCVXGBoost"] = np.std(cv_scores_xg)

# Calculate squared log error for each row and add a new column
squared_log_errors = (np.log1p(X_train_final["PredictedValuesXGBoost"]) - np.log1p(y_train))**2
X_train_final["SquaredLogErrorXGBoost"] = squared_log_errors

# Print or display the results
print("Training Set Predictions:")
print(results_df.head())
X_train_final


Training Set Predictions:
   ActualValues  PredictedValuesLightGBM
0        208500            201412.688312
1        181500            172649.796284
2        223500            216089.583937
3        140000            164844.080361
4        250000            286599.782010


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,PredictedValuesXGBoost,MeanCVXGBoost,StdCVXGBoost,SquaredLogErrorXGBoost
0,60.0,RL,65.0,8450.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,0.0,2.0,2008.0,WD,Normal,208300.265625,0.126755,0.024401,9.192499e-07
1,20.0,RL,80.0,9600.0,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,Shed,0.0,5.0,2007.0,WD,Normal,181966.140625,0.126755,0.024401,6.581072e-06
2,60.0,RL,68.0,11250.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,Shed,0.0,9.0,2008.0,WD,Normal,223013.078125,0.126755,0.024401,4.757300e-06
3,70.0,RL,60.0,9550.0,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,Shed,0.0,2.0,2006.0,WD,Abnorml,141033.875000,0.126755,0.024401,5.414078e-05
4,60.0,RL,84.0,14260.0,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,Shed,0.0,12.0,2008.0,WD,Normal,253601.968750,0.126755,0.024401,2.046364e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,0.0,8.0,2007.0,WD,Normal,174096.906250,0.126755,0.024401,2.676708e-05
1456,20.0,RL,85.0,13175.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,0.0,2.0,2010.0,WD,Normal,207899.640625,0.126755,0.024401,1.010376e-04
1457,70.0,RL,66.0,9042.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,2500.0,5.0,2010.0,WD,Normal,265223.718750,0.126755,0.024401,2.304505e-05
1458,20.0,RL,68.0,9717.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,Shed,0.0,4.0,2010.0,WD,Normal,141243.015625,0.126755,0.024401,3.875467e-05


In [23]:
import lightgbm as lgb

# Split the data into training and validation sets
X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a column transformer to handle categorical encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Transform the training data
X_train2_transformed = preprocessor.fit_transform(X_train2)
X_train_transformed = preprocessor.fit_transform(X_train)


model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=2000,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.25,
                              feature_fraction_seed=9, bagging_seed=9, seed=42)

# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores_lgb = cross_val_score(model_lgb, X_train_transformed, y_train, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores_lgb)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores_lgb))
print("Std CV Score: ", np.std(cv_scores_lgb))


[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000935 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1377
[LightGBM] [Info] Number of data point

[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_f

In [25]:
# Fit the model on the entire training set
model_lgb.fit(X_train_transformed, y_train)


# Add a new column to the training set with predicted values
X_train_final["PredictedValuesLightGBM"] = model_lgb.predict(X_train_transformed)
X_train_final["MeanCVLightGBM"] = np.mean(cv_scores_lgb)
X_train_final["StdCVLightGBM"] = np.std(cv_scores_lgb)

# Calculate squared log error for each row and add a new column
squared_log_errors = (np.log1p(X_train_final["PredictedValuesLightGBM"]) - np.log1p(y_train))**2
X_train_final["SquaredLogErrorLightGBM"] = squared_log_errors


# Print or display the results
print("Training Set Predictions:")
print(results_df.head())
X_train_final

[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001209 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1396
[LightGBM] [Info] Number of data point

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,SaleType,SaleCondition,PredictedValuesXGBoost,MeanCVXGBoost,StdCVXGBoost,SquaredLogErrorXGBoost,PredictedValuesLightGBM,MeanCVLightGBM,StdCVLightGBM,SquaredLogErrorLightGBM
0,60.0,RL,65.0,8450.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,208300.265625,0.126755,0.024401,9.192499e-07,203671.382339,0.125349,0.02033,0.000549
1,20.0,RL,80.0,9600.0,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,WD,Normal,181966.140625,0.126755,0.024401,6.581072e-06,176434.466699,0.125349,0.02033,0.000801
2,60.0,RL,68.0,11250.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,WD,Normal,223013.078125,0.126755,0.024401,4.757300e-06,218400.291828,0.125349,0.02033,0.000533
3,70.0,RL,60.0,9550.0,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,WD,Abnorml,141033.875000,0.126755,0.024401,5.414078e-05,154234.859740,0.125349,0.02033,0.009377
4,60.0,RL,84.0,14260.0,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,WD,Normal,253601.968750,0.126755,0.024401,2.046364e-04,270434.309576,0.125349,0.02033,0.006173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,174096.906250,0.126755,0.024401,2.676708e-05,173294.744477,0.125349,0.02033,0.000096
1456,20.0,RL,85.0,13175.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,207899.640625,0.126755,0.024401,1.010376e-04,200768.709990,0.125349,0.02033,0.002021
1457,70.0,RL,66.0,9042.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,265223.718750,0.126755,0.024401,2.304505e-05,262465.338643,0.125349,0.02033,0.000233
1458,20.0,RL,68.0,9717.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,WD,Normal,141243.015625,0.126755,0.024401,3.875467e-05,140095.421897,0.125349,0.02033,0.000207


In [26]:
# Split the data into training and validation sets
X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a column transformer to handle categorical encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Transform the training data
X_train2_transformed = preprocessor.fit_transform(X_train2)
X_train_transformed = preprocessor.fit_transform(X_train)


random_forest = RandomForestRegressor(criterion='squared_error', max_depth=23, random_state=42)

# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores_rf = cross_val_score(random_forest, X_train_transformed, y_train, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores_rf)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores))
print("Std CV Score: ", np.std(cv_scores))

Cross-validation scores:  [0.13647646 0.16265677 0.11293399 0.14224117 0.16739923 0.17278652
 0.16708395 0.13200163 0.14218179 0.09971584]
Mean CV Score:  0.1253486211561304
Std CV Score:  0.02033037957856136


In [27]:
# Fit the model on the entire training set
random_forest.fit(X_train_transformed, y_train)

# Transform the validation data
X_valid_transformed = preprocessor.transform(X_valid)

# Predict on the validation set
y_pred_valid = random_forest.predict(X_valid_transformed)

# Calculate the neg_mean_squared_log_error on the validation set
validation_score_rf = mean_squared_log_error(y_valid, y_pred_valid)
print("Validation Score (neg_mean_squared_log_error): ", validation_score_rf)

# Add a new column to the training set with predicted values
X_train_final["PredictedValuesRandomForest"] = random_forest.predict(X_train_transformed)
X_train_final["MeanCVRandomForest"] = np.mean(cv_scores)
X_train_final["StdCVRandomForest"] = np.std(cv_scores)

# Calculate squared log error for each row and add a new column
squared_log_errors = (np.log1p(X_train_final["PredictedValuesRandomForest"]) - np.log1p(y_train))**2
X_train_final["SquaredLogErrorRandomForest"] = squared_log_errors

X_train_final

Validation Score (neg_mean_squared_log_error):  0.0037508865993548493


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,StdCVXGBoost,SquaredLogErrorXGBoost,PredictedValuesLightGBM,MeanCVLightGBM,StdCVLightGBM,SquaredLogErrorLightGBM,PredictedValuesRandomForest,MeanCVRandomForest,StdCVRandomForest,SquaredLogErrorRandomForest
0,60.0,RL,65.0,8450.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.024401,9.192499e-07,203671.382339,0.125349,0.02033,0.000549,205523.466667,0.125349,0.02033,0.000207
1,20.0,RL,80.0,9600.0,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,0.024401,6.581072e-06,176434.466699,0.125349,0.02033,0.000801,175070.320000,0.125349,0.02033,0.001301
2,60.0,RL,68.0,11250.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0.024401,4.757300e-06,218400.291828,0.125349,0.02033,0.000533,220379.983333,0.125349,0.02033,0.000198
3,70.0,RL,60.0,9550.0,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,0.024401,5.414078e-05,154234.859740,0.125349,0.02033,0.009377,159469.780000,0.125349,0.02033,0.016955
4,60.0,RL,84.0,14260.0,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,0.024401,2.046364e-04,270434.309576,0.125349,0.02033,0.006173,265255.930000,0.125349,0.02033,0.003509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.024401,2.676708e-05,173294.744477,0.125349,0.02033,0.000096,176388.504786,0.125349,0.02033,0.000062
1456,20.0,RL,85.0,13175.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.024401,1.010376e-04,200768.709990,0.125349,0.02033,0.002021,206899.500000,0.125349,0.02033,0.000221
1457,70.0,RL,66.0,9042.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.024401,2.304505e-05,262465.338643,0.125349,0.02033,0.000233,258720.270000,0.125349,0.02033,0.000878
1458,20.0,RL,68.0,9717.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.024401,3.875467e-05,140095.421897,0.125349,0.02033,0.000207,143048.815833,0.125349,0.02033,0.000042


In [28]:
for col in categorical_cols:
    # Find the most frequent value in X_train for the current column
    most_frequent_value = X_train[col].mode()[0]

    # Replace values in X_test with the most frequent value if not in X_train
    X_test[col] = X_test[col].apply(lambda x: x if x in X_train[col].unique() else most_frequent_value)
    
X_test_transformed = preprocessor.fit_transform(X_test)


In [61]:
X_test_transformed

<1459x270 sparse matrix of type '<class 'numpy.float64'>'
	with 97930 stored elements in Compressed Sparse Row format>

In [63]:
len(X_train_transformed[0])


298

In [29]:
# Apply one-hot encoding to the test set
X_test_transformed = pd.get_dummies(X_test, columns=categorical_cols)
X_train_transformed = pd.get_dummies(X_train, columns=categorical_cols)

# Ensure the same columns are present in both X_train_transformed and X_test_transformed
common_columns = X_train_transformed.columns.intersection(X_test_transformed.columns)
X_test_transformed = X_test_transformed[common_columns]

In [30]:
# Find missing columns in X_test_transformed
missing_columns = set(X_train_transformed.columns) - set(X_test_transformed.columns)

# Add missing columns to X_test_transformed and fill with zeros
for col in missing_columns:
    X_test_transformed[col] = 0
    
# Ensure the same columns are present in both X_train_transformed and X_test_transformed
common_columns = X_train_transformed.columns.intersection(X_test_transformed.columns)

# Get the feature names from the training set
feature_names = X_train_transformed[common_columns].columns

# Reorder columns in the test set to match the order of feature names from the training set
X_test_transformed = X_test_transformed[feature_names]

In [31]:
X_test_transformed

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20.0,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,144.0,...,0,0,0,1,0,0,0,0,1,0
1,20.0,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,60.0,74.0,13830.0,5.0,5.0,1997.0,1998.0,0.0,791.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,60.0,78.0,9978.0,6.0,6.0,1998.0,1998.0,20.0,602.0,0.0,...,0,0,0,1,0,0,0,0,1,0
4,120.0,43.0,5005.0,8.0,5.0,1992.0,1992.0,0.0,263.0,0.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160.0,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1455,160.0,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,0.0,...,0,0,0,1,1,0,0,0,0,0
1456,20.0,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,0.0,...,0,0,0,1,1,0,0,0,0,0
1457,85.0,62.0,10441.0,5.0,5.0,1992.0,1992.0,0.0,337.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [32]:
X_train_transformed

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,0,0,0,1,1,0,0,0,0,0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1456,20.0,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,790.0,163.0,...,0,0,0,1,0,0,0,0,1,0
1457,70.0,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,275.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1458,20.0,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,49.0,1029.0,...,0,0,0,1,0,0,0,0,1,0


In [44]:
from sklearn.linear_model import LinearRegression

# Step 3: Generate Meta-Features
meta_features_train = np.column_stack([
    random_forest.predict(X_train_transformed),
    xg_reg.predict(X_train_transformed),
    model_lgb.predict(X_train_transformed),
])

meta_features_test = np.column_stack([
    random_forest.predict(X_test_transformed),
    xg_reg.predict(X_test_transformed),
    model_lgb.predict(X_test_transformed),
])

# Combine original features with meta-features
X_train_stacked = np.column_stack([X_train_transformed, meta_features_train])
X_test_stacked = np.column_stack([X_test_transformed, meta_features_test])

# Step 4: Train the Final Estimator
final_estimator = Ridge()


# Create a custom scorer for neg_mean_squared_log_error
scorer = make_scorer(score_func=lambda y, y_pred: -1.0 * np.sqrt(mean_squared_log_error(y, y_pred)), greater_is_better=False)

# Create 5-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores_ridge = cross_val_score(final_estimator, X_train_transformed, y_train, cv=kf, scoring=scorer)

# Print the cross-validation scores
print("Cross-validation scores: ", cv_scores_ridge)

# Print the mean and standard deviation of the cross-validation scores
print("Mean CV Score: ", np.mean(cv_scores_ridge))
print("Std CV Score: ", np.std(cv_scores_ridge))

C:\Users\germa\anaconda3\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
C:\Users\germa\anaconda3\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Cross-validation scores:  [0.14645939 0.20311058 0.13512135 0.18538666 0.1571407  0.22187339
 0.17112188 0.13372303 0.14883809 0.11346801]
Mean CV Score:  0.1616243069229312
Std CV Score:  0.03196779317799662


In [ ]:
final_estimator.fit(X_train_stacked, y_train)

# Step 5: Make Predictions on Test Data
final_predictions = final_estimator.predict(X_test_stacked)

final_predictions

In [38]:
print(len(random_forest.predict(X_test_transformed)))
print(len(X_test_final["MeanCVRandomForest"]))
# Repeat for other arrays...

# Check the shapes of the arrays
print(random_forest.predict(X_test_transformed).shape)
print(X_test_final["MeanCVRandomForest"].shape)

1459


C:\Users\germa\anaconda3\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


NameError: name 'X_test_final' is not defined

In [39]:
X_test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20.0,RH,80.0,11622.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,120.0,0.0,Ex,MnPrv,Shed,0.0,6.0,2010.0,WD,Normal
1,20.0,RL,81.0,14267.0,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,0.0,0.0,Ex,MnPrv,Gar2,12500.0,6.0,2010.0,WD,Normal
2,60.0,RL,74.0,13830.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,3.0,2010.0,WD,Normal
3,60.0,RL,78.0,9978.0,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,6.0,2010.0,WD,Normal
4,120.0,RL,43.0,5005.0,Pave,Grvl,IR1,HLS,AllPub,Inside,...,144.0,0.0,Ex,MnPrv,Shed,0.0,1.0,2010.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160.0,RM,21.0,1936.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,6.0,2006.0,WD,Normal
1455,160.0,RM,21.0,1894.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,4.0,2006.0,WD,Abnorml
1456,20.0,RL,160.0,20000.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,0.0,9.0,2006.0,WD,Abnorml
1457,85.0,RL,62.0,10441.0,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0.0,0.0,Ex,MnPrv,Shed,700.0,7.0,2006.0,WD,Normal


In [83]:
salida = pd.DataFrame({'Id': test_ids, 'SalePrice': final_predictions})
salida.to_csv("Prueba8.csv", index=False)